# Import libraries

In [ ]:
import os
import copy
import matplotlib
import pylab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
from lmfit import  Model, Parameters

# Functions

In [ ]:
style_param = {'legend.fontsize': '20',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '26',
         'axes.titlesize':'26',
         'xtick.labelsize':'20',
         'ytick.labelsize':'20',
         'xtick.major.pad':'16',
         'ytick.major.pad':'16'}
    
pylab.rcParams.update(style_param)

In [ ]:
#definition of linear-quadratic model
def lq(x, alpha, beta):
    return (-alpha*x-beta*x**2)

# Open data

In [ ]:
fname = os.path.join('tmp','rawdata.h5')
df = pd.read_hdf(fname, 'data')

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.set_index(['article','energy','nr'],inplace=True)

# Fit

In [ ]:
lqmodel = Model(lq, nanpolicy='propagate')
params = Parameters()
params.add('alpha', value=0.0002, min=0)
params.add('beta', value=0.0001, min=0, max=1e8)

In [ ]:
for name, group in df.groupby('article'):
    for energy, data in group.groupby('energy'):
        
        result = lqmodel.fit(params=params, 
                             data=np.log(data.sf), 
                             x=data.dose,
                             #weight is the inverse of the logarithm difference of the upper and lower error bars
                             weights=np.fabs(1/(np.log(data.sf_err_up.values)-np.log(data.sf_err_down.values)))
                            )
        
        df.loc[(name,energy),'alpha_fit'] = result.values['alpha']
        df.loc[(name,energy),'alpha_fit_err'] = result.params['alpha'].stderr
        df.loc[(name,energy),'beta_fit_err'] = result.params['beta'].stderr
        df.loc[(name,energy),'residuals']=(result.residual)
#        df.loc[(name,energy),'residuals_limit']=(result.residual.std())
        df.loc[(name,energy),'covar_fit']=result.covar[1][0]
        df.loc[(name,energy),'fit_results']=copy.deepcopy(result)
        
        #if beta is less than 0.01 or the beta error is greater than the value of the beta parameter, 
        #then we take the value of beta and the error to be 0 
        #so that the beta distribution does not contain negative values
        if (result.values['beta']<result.params['beta'].stderr).any() & (result.values['beta']<0.01).any():
            df.loc[(name,energy),'beta_fit'] = 0.0
            #df.loc[(name,energy),'beta_fit_err'] = 0.0

        else:
            df.loc[(name,energy),'beta_fit'] = result.values['beta']

In [ ]:
df.sort_index(inplace=True)

In [ ]:
#choose the experiment to visualize the results from article 10, 
#where a proton energy of 5010keV was used

data = df.loc['10'].loc[5010]
fig,ax = plt.subplots(figsize = (10,3))

error = np.stack((data.sf-data.sf_err_down,data.sf_err_up-data.sf))

#err up err down for experimental data points
ax.errorbar(data.dose,  
             data.sf - np.exp(data.fit_results.iloc[0].best_fit),  
             yerr=error,
             fmt='o', capsize=5,color='k')

ax.axhline(0., color='k')

ax.set_ylabel("Residuals, "+ r'$\rho \;[-]$')
ax.set_xlabel("")

ax.set_xlim(0.,5.5)
ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(data.dose,
                          data.sf - np.exp(data.fit_results.iloc[0].best_fit),
                          error)),
                          columns=['dose','residuals', 'residuals_err'])
tmp.to_csv("results/figure_5_1a.csv")
fig.savefig(fname="results/figure_5_1a.png",dpi= 700)

In [ ]:
#choose the experiment to visualize the results from article 10, 
#where a proton energy of 5010keV was used

data = df.loc['10'].loc[5010]
fig,ax = plt.subplots(figsize = (10,6))

ax.scatter(data.dose, 
            data.sf,
            s=6, c="k")

ax.errorbar(data.dose, 
             data.sf, 
             yerr=error,
             fmt='o', color='k', capthick=2,capsize=5, 
             label = "experimental data")

ax.plot(np.append(0,data.dose), 
         np.append(1,np.exp(data.fit_results.iloc[0].best_fit)), 
         c='k', 
         label="LQ model")

ax.set_xlim(0.,5.5)
ax.set_ylabel("Surviving Fraction, S [-]")
ax.set_xlabel("Dose [Gy]")

ax.set_yscale('log')

leg = plt.legend(loc='best', ncol=1)         
leg.get_frame().set_alpha(0.5)

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(data.dose,
                          data.sf)),
                          columns=['dose','sf'])
tmp.to_csv("results/figure_5_1b.csv")
fig.savefig(fname="results/figure_5_1b.png",dpi= 700)

# Definition of RBE

In [ ]:
a1 = df.loc["10"].loc[0].fit_results[0].params['alpha'].value
b1 = df.loc["10"].loc[0].fit_results[0].params['beta'].value

a2 = df.loc["10"].loc[3200].fit_results[0].params['alpha'].value
b2 = df.loc["10"].loc[3200].fit_results[0].params['beta'].value

In [ ]:
dose1 = np.append(0 , df.loc["10"].loc[0].dose.values)
dose2 = np.append(0 , df.loc["10"].loc[3200].dose.values)

sf1 = np.exp(-a1*dose1-b1*dose1**2)
sf2 = np.exp(-a2*dose2-b2*dose2**2)

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

ax.plot(dose1, 
         sf1, 
         "k",
         label="200 kVp RTG",
         )
ax.plot(dose2,
         sf2,
         "k--",
         label = "3.2 MeV protons")

ax.axhline(0.5,c="k" ,xmax=0.69,ls=':',lw=1.2)
ax.text(0.1, .52, '50% of S',c='k')

ax.text(-0.25, .48, '0.5')

ax.axvline(2.75,c="k", ymax=0.72,ls=':',lw=1.2)
ax.text(2.85, .1, '2.75 Gy\n',c='k')

ax.axvline(1.65,c="k", ymax=0.72,ls=':',lw=1.2)
ax.text(1.75, .1, '1.65 Gy\n',c='k')

ax.set_xlabel("Dose [Gy]")
ax.set_ylabel("Surviving Fraction, S[-]")

ax.set_yscale('log')
ax.set_xlim(0,4)
ax.set_ylim(0.08,1)

ax.legend( loc='upper right', ncol=1,  borderaxespad=0.)
ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')


In [ ]:
tmp=pd.DataFrame(list(zip(dose2,sf2,
                          dose1,sf1)),
                          columns=['dose_proton','sf_proton','dose_ref','sf_ref'])
tmp.to_csv("results/figure_2_1.csv")

fig.savefig(fname="results/figure_2_1.png",dpi= 700)

# Compare alfa & beta

In [ ]:
i=0
for name, group in df.groupby('article'):
    for energy, data in group.groupby('energy'):
        i=i+1
        if data['beta_fit'].any() == 0.:
            df.loc[(name,energy),'div_b_bfit'] = 0.
            df.loc[(name,energy),'err_b_bfit'] = 0.
        else:
            df.loc[(name,energy),'div_b_bfit'] = data.beta/data.beta_fit

        df.loc[(name,energy),'err_b_bfit'] = (data.beta_err/data.beta_fit)-(data.beta*data.beta_fit_err)/(data.beta_fit**2)
        
        df.loc[(name,energy),'div_a_afit'] = data.alpha/data.alpha_fit
        df.loc[(name,energy),'err_a_afit'] = (data.alpha_err/data.alpha_fit)-(data.alpha*data.alpha_fit_err)/(data.alpha_fit**2)
#assigning unique identifiers to subsequent experiments 
        df.loc[(name,energy),'exp_no'] = i

In [ ]:
df2=df.reset_index().drop_duplicates(subset=['article','energy'])

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

ax.plot(df2.exp_no,
         df2.div_a_afit,
         'o', alpha = 0.6)
ax.errorbar(df2.exp_no,
             df2.div_a_afit,
             yerr=df2.err_a_afit,
             zorder=2, fmt='o',
             capsize=5,color='k')

ax.hlines(y=1.2, xmin=0, xmax=len(df2.exp_no.values), linewidth=2, color='k')
ax.hlines(y=0.8, xmin=0, xmax=len(df2.exp_no.values), linewidth=2, color='k')

ax.set_yticks(np.arange(0,2,0.2))
ax.set_ylim(0,2)
ax.set_xlim(0,35)

ax.set_ylabel(r'$\alpha_{P}/ \alpha_{J} \;[-]$')
ax.set_xlabel("experiment number")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(df2.exp_no,
                          df2.div_a_afit,
                          df2.err_a_afit)),
                          columns=['exp_no','alpha_alpha_ref','alpha_alpha_ref_err'])
tmp.to_csv("results/figure_5_3a.csv")

fig.savefig(fname="results/figure_5_3a.png",dpi= 700)

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

ax.plot(df2.exp_no,
         df2.div_b_bfit,
         'o', alpha = 0.6)
ax.errorbar(df2.exp_no,
             df2.div_b_bfit,
             yerr=df2.err_b_bfit,
             zorder=2, fmt='o',
             capsize=5,color='k')

ax.hlines(y=1.2, xmin=0, xmax=len(df2.exp_no.values), linewidth=2, color='k')
ax.hlines(y=0.8, xmin=0, xmax=len(df2.exp_no.values), linewidth=2, color='k')

ax.set_ylim(0,2)
ax.set_xlim(0,35)

ax.set_ylabel(r'$\beta_{P}/ \beta_{J} \;[-]$')
ax.set_xlabel("experiment number")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(df2.exp_no,
                          df2.div_b_bfit,
                          df2.err_b_bfit)),
                          columns=['exp_no','beta_beta_ref','beta_beta_ref_err'])
tmp.to_csv("results/figure_5_3b.csv")

fig.savefig(fname="results/figure_5_3b.png",dpi= 700)

# Save temporary files

In [ ]:
df2 = df[["alpha","beta","beta_err","alpha_err","alpha_fit","alpha_fit_err","beta_fit","beta_fit_err","sf","sf_err_up","sf_err_down","covar_fit","LET"]]

In [ ]:
hdf_fname = os.path.join('tmp','fitted_data.h5')
df2.to_hdf(hdf_fname, 'data', format='table')